In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
from sklearn.model_selection import train_test_split

In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

## for ['gender', city]----OHE
## for cough --> OrdinalEncoding
## for fever ---> SimpleImputer

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=.2)
X_train.shape, X_test.shape

((80, 5), (20, 5))

# Without column Transformer

In [9]:
## apply simpleImputer
si = SimpleImputer()
X_train_si = si.fit_transform(X_train[['fever']])

X_test_si = si.transform(X_test[['fever']])
X_test_si

array([[101.        ],
       [100.        ],
       [102.        ],
       [100.        ],
       [101.05555556],
       [101.        ],
       [103.        ],
       [104.        ],
       [101.05555556],
       [101.        ],
       [ 98.        ],
       [ 98.        ],
       [100.        ],
       [ 98.        ],
       [100.        ],
       [ 98.        ],
       [ 99.        ],
       [ 98.        ],
       [101.        ],
       [ 98.        ]])

In [10]:
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])
X_test_cough = oe.transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [13]:
ohe = OneHotEncoder(drop='first')
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']]).toarray()
X_test_gender_city = ohe.transform(X_test[['gender', 'city']]).toarray()

X_train_gender_city

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 1.],
       [1., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 1., 0.],
       [1., 1., 0., 0.],
       [1., 1., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],


In [14]:
X_train_age = X_train.drop(columns=['gender', 'fever', 'cough', 'city']).values
X_test_age = X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

In [16]:
X_train_age.shape

(80, 1)

In [17]:
X_train_transformed = np.concatenate((X_train_age, X_train_si, X_train_gender_city, X_train_cough), axis=1)
X_test_transformed = np.concatenate((X_test_age, X_test_si, X_test_gender_city, X_test_cough), axis=1)

In [18]:
X_train_transformed.shape

(80, 7)

In [21]:
X_train_transformed

array([[ 81.        , 101.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 64.        , 102.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 23.        , 101.05555556,   1.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 33.        , 102.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   1.        ],
       [ 10.        ,  98.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   1.        ],
       [ 83.        , 101.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 34.        , 101.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   1.        ],
       [ 20.        , 101.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 60.        ,  99.        ,   0.        ,   0.        ,
          0.    

## Do it Use Column Transform

In [22]:
from sklearn.compose import ColumnTransformer

In [25]:
df.head(1)

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No


In [26]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(), ['fever']),
    ('tnf2', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city']),
    ('tnf3', OrdinalEncoder(categories=[["Mild", 'Strong']]), ['cough'])
    ], remainder='passthrough')

In [29]:
transformer.fit_transform(X_train).shape
transformer.transform(X_test).shape

(20, 7)